In [ ]:
#크롤링
import requests
from bs4 import BeautifulSoup

url='https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL'

# 54페이지부터 시작
#total=[]

date_list=[]
num_list=[]
all_dict={}

for page in range(55,517):
    params={
        'page':page
    }
    resp=requests.get(url,params=params)    
    soup=BeautifulSoup(resp.content,'html.parser')

    # 그 페이지에 있는 날짜 저장.
    
    all_date=soup.find_all(class_='date')
    for d in range(len(all_date)):
        date_list.append(all_date[d].text.strip())
    #    print(all_date[d].text.strip())
    
    # 종가 찾아서 저장하기.
    all_num=soup.find_all(class_='num')
    for n in range(len(all_num)):
        if n%3==0:
            num_list.append(all_num[n].text.strip())
            #print(all_num[n].text.strip())
    

    for j in range(len(date_list)):
        all_dict[date_list[j]]=num_list[j]    
    print(all_dict)
       
        #if page%50==0:
         #   print(page)
    #total.append(all_dict)


In [ ]:
print(date_list[:3])
print(date_list[-2:])

In [ ]:
print(num_list[:5])
print(num_list[-5:])

In [ ]:
len(date_list)

In [ ]:
len(num_list)

In [ ]:
len(all_dict)

In [ ]:
all_dict

### 해당되지 않는 날짜 앞,뒤  지워주기

In [ ]:
del date_list[:3]
del date_list[-2:]

del num_list[:3]
del num_list[-2:]
#['2018.01.04', '2018.01.03', '2018.01.02']

del all_dict['2004.12.31']
del all_dict['2004.12.30']
del all_dict['2018.01.04']
del all_dict['2018.01.03']
del all_dict['2018.01.02']
#el all_dict['2004.12.24']

In [ ]:
len(num_list)

In [ ]:
len(date_list)

In [ ]:
len(all_dict)

In [ ]:
#all_dict

### 크롤링해온것 일단 파일에 저장

In [ ]:
import pandas as pd

df_c=pd.DataFrame()
df_c['date']=date_list
df_c['콜금리']=num_list
df_c['date']=pd.to_datetime(df_c['date'])
df_c=df_c.set_index('date')
df_c

In [ ]:
df_c.to_csv("C:\\Users\\Jungsk\\Downloads\\멀캠_혁신성장\\crwal_call_rate.csv",sep=' ')

In [ ]:
test_csv=pd.read_csv("C:\\Users\\Jungsk\\Downloads\\멀캠_혁신성장\\crwal_call_rate.csv",sep=' ')
test_csv

### 실제 달력 날짜 가져오기

In [ ]:
import pandas as pd
import datetime
cal=pd.Series(pd.date_range(start='2005.01.01',end='2017.12.31',freq='D'))
cal=[d for d in reversed(cal)]
cal

In [ ]:
len(cal)

## Data Frame에 넣기

###  df :  크롤링한 date & call금리 저장

In [ ]:
df=pd.DataFrame()
df['date']=date_list
df['콜금리']=num_list
df['date']=pd.to_datetime(df['date'])
df

### df 1: 실제 날짜 저장

In [ ]:
df1=pd.DataFrame()
df1['date']=cal
df1

### df 와 df1 을 Merge

In [ ]:
df2=pd.merge(df1,df,how='outer',on='date')

df2=df2.fillna(method='ffill')
if df2.isnull().values.any():
    df2=df2.fillna(method='bfill')

df2

### 인덱스를 날짜로

In [ ]:
df2['date']=pd.to_datetime(df2['date'])
df2

In [ ]:
df2=df2.set_index('date')
df2

### 1 달 전 날짜 구해놓기 - 리스트에 저장

In [ ]:
from datetime import datetime,timedelta

month_ago=df2['Before 1M']=df2.index-pd.DateOffset(months=1)
df2

### '현재 날짜'로 , '1 달 전 날짜' 를 구해서, 데이터프레임 키값으로 넣어주기...!!


In [ ]:
key_date=[]
current_date=[]

for i in range(len(cal)):
    before=df2['Before 1M'][i]
    before_str=str(before)
    key_date.append(before_str[:10])
    before_str[-10]
key_date

In [ ]:
len(key_date)

### 이전 날짜를 이용해 1 달 전 call금리를 가져온다.

In [ ]:
ago_call_list=[]

try:
    for i in range(len(cal)):
    
        before=df2.loc[key_date[i]]
        b=float(before.iloc[0][0])
        ago_call_list.append(b)
        
        #print(q)
except:
    pass
ago_call_list

In [ ]:
len(ago_call_list)

In [ ]:
len(cal)

### 둘이 길이 맞아야 함. cal, ago_Call_list

In [ ]:
#len(ago_call_list) 
for i in range(0,31): 
    ago_call_list.append('0')

print(len(ago_call_list))

In [ ]:
len(num_list)

In [ ]:
date_list[20:]

In [ ]:
num_list[20:]

### df3: date 와 1 달 전 콜금리 저장

In [ ]:
df3=pd.DataFrame()
df3['date']=cal
df3['1달 전 콜금리']=ago_call_list
df3['date']=pd.to_datetime(df3['date'])
df3

### df4 : merge   df2, df3

In [ ]:
df4=pd.merge(df2,df3,how='outer',on='date')

df4=df4.fillna(method='ffill')
if df4.isnull().values.any():
    df4=df4.fillna(method='bfill')

df4

### 상승/하락 정하기

In [ ]:
len(ago_call_list)

### 현재 날짜 가져오기.

In [ ]:
current_date=df2.index
current_date_list=[]

for i in range(len(cal)):
    c_date=current_date[i]
    str_c=str(c_date)
    current_date_list.append(str_c[:10])
current_date_list

### 현재 날짜로 현재 콜금리 해서

In [ ]:
now_call_list=[]
try:
    for i in range(len(cal)):
    
        now_call=df2.loc[current_date_list[i]]
        n=float(now_call.iloc[0][0])
        now_call_list.append(n)
        
        #print(q)
except:
    pass
now_call_list

In [ ]:
result=[]
value=0.0
label=[]


#def 콜금리계산():
for i in range(0,len(ago_call_list)):
    value=float(now_call_list[i])-float(ago_call_list[i])
    
    if value>0 :
        ap='+'+str(value)+'%'
        result.append(ap)
        label.append('1')
    elif value<0:
        ap='-'+str(value)+'%'
        result.append(ap)
        label.append('-1')
    else:
        result.append('0.00%')
        label.append('0')
    
result.append('0')
result

In [ ]:
label

In [ ]:
len(label)

In [ ]:
len(result)

In [ ]:
result[-10:]

In [ ]:
del result[-1]

In [ ]:
len(cal)

### 길이 맞추기.- result 와 cal 사이

for i in range(0,30):
    result.append('-')
print(len(result))
print(result)

for i in range(0,51):
    result.append('-')
print(len(result))
print(result)

### df5 : date  전달대비 

In [ ]:
df5=pd.DataFrame()
df5['date']=cal
df5['전달대비']=result
df5['label']=label
df5

In [ ]:
df4

### df6 : df5, df4  Merge

In [ ]:
df6=pd.merge(df4,df5,how='outer',on='date')

df6=df6.fillna(method='ffill')
if df6.isnull().values.any():
    df6=df6.fillna(method='bfill')

df6=df6.set_index('date')
df6
    

### 파일에 저장

In [ ]:
df6.to_csv("C:\\Users\\Jungsk\\Downloads\\멀캠_혁신성장\\call_rate.csv",sep=' ')


In [ ]:
test_read=pd.read_csv("C:\\Users\\Jungsk\\Downloads\\멀캠_혁신성장\\call_rate.csv",sep=' ')
test_read